In [9]:
# path: ~/ml/catdog

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

img = load_img('./train/cats/cat.0.jpg')  # this is a PIL image
x = img_to_array(img)  # this is a Numpy array with shape (374, 500, 3)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 374, 500, 3)

# the .flow() command below generates batches of randomly transformed images
# and saves the results to the `preview/` directory
i = 0
for batch in datagen.flow(x, batch_size=1,
                          save_to_dir='preview', save_prefix='cat', save_format='jpeg'):
    i += 1
    if i > 20:
        break  # otherwise the generator would loop indefinitely

In [12]:
x.shape

(1, 374, 500, 3)

In [14]:
y = img_to_array(load_img('./train/cats/cat.0.jpg'))
y.shape

(374, 500, 3)

In [17]:
'''This script goes along the blog post
"Building powerful image classification models using very little data"
from blog.keras.io.
It uses data that can be downloaded at:
https://www.kaggle.com/c/dogs-vs-cats/data
In our setup, we:
- created a catdog/ folder
- created train/ and validation/ subfolders inside data/
- created cats/ and dogs/ subfolders inside train/ and validation/
- put the cat pictures index 0-999 in data/train/cats
- put the cat pictures index 1000-1400 in data/validation/cats
- put the dogs pictures index 12500-13499 in data/train/dogs
- put the dog pictures index 13500-13900 in data/validation/dogs
So that we have 1000 training examples for each class, and 400 validation examples for each class.
In summary, this is our directory structure:
```
catdog/
    train/
        dogs/
            dog001.jpg
            dog002.jpg
            ...
        cats/
            cat001.jpg
            cat002.jpg
            ...
    validation/
        dogs/
            dog001.jpg
            dog002.jpg
            ...
        cats/
            cat001.jpg
            cat002.jpg
            ...
```
'''

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K


# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = './train'
validation_data_dir = './validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

model.save_weights('first_try.h5')

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.
Epoch 1/50
125/125 [==============================] - 48s - loss: 0.7088 - acc: 0.5325 - val_loss: 0.6810 - val_acc: 0.6012
Epoch 2/50
125/125 [==============================] - 42s - loss: 0.6942 - acc: 0.5925 - val_loss: 0.6312 - val_acc: 0.6012
Epoch 3/50
125/125 [==============================] - 44s - loss: 0.6439 - acc: 0.6295 - val_loss: 0.6300 - val_acc: 0.6312
Epoch 4/50
125/125 [==============================] - 40s - loss: 0.6458 - acc: 0.6290 - val_loss: 0.5853 - val_acc: 0.6950
Epoch 5/50
125/125 [==============================] - 52s - loss: 0.6385 - acc: 0.6615 - val_loss: 0.6075 - val_acc: 0.6925
Epoch 6/50
125/125 [==============================] - 48s - loss: 0.6225 - acc: 0.6550 - val_loss: 0.5634 - val_acc: 0.7063
Epoch 7/50
125/125 [==============================] - 48s - loss: 0.6288 - acc: 0.6520 - val_loss: 0.5540 - val_acc: 0.7311
Epoch 8/50
125/125 [===========================

In [18]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
activation_6 (Activation)    (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
activation_7 (Activation)    (None, 72, 72, 32)        0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 34, 34, 64)        18496     
__________

In [ ]:

'''Train a simple deep CNN on the CIFAR10 small images dataset.
GPU run command with Theano backend (with TensorFlow, the GPU is automatically used):
    THEANO_FLAGS=mode=FAST_RUN,device=gpu,floatx=float32 python cifar10_cnn.py
It gets down to 0.65 test logloss in 25 epochs, and down to 0.55 after 50 epochs.
(it's still underfitting at that point, though).
'''

import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

import os

batch_size = 32
num_classes = 10
epochs = 200
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        steps_per_epoch=x_train.shape[0] // batch_size,
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)

# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
Using real-time data augmentation.
Epoch 1/200
1562/1562 [==============================] - 131s - loss: 1.8248 - acc: 0.3243 - val_loss: 1.5924 - val_acc: 0.4176
Epoch 2/200
1562/1562 [==============================] - 138s - loss: 1.5486 - acc: 0.4327 - val_loss: 1.4077 - val_acc: 0.4896
Epoch 3/200
1562/1562 [==============================] - 132s - loss: 1.4285 - acc: 0.4825 - val_loss: 1.2547 - val_acc: 0.5503
Epoch 4/200
1562/1562 [==============================] - 144s - loss: 1.3384 - acc: 0.5216 - val_loss: 1.1590 - val_acc: 0.5906
Epoch 5/200
1562/1562 [==============================] - 123s - loss: 1.2692 - acc: 0.5488 - val_loss: 1.0915 - val_acc: 0.6097
Epoch 6/200
1562/1562 [==============================] - 124s - loss: 1.2101 - acc: 0.5727 - val_loss: 1.0253 - val_acc: 0.6409
Epoch 7/200
1562/1562 [==============================] - 132s - loss: 1.1507 - acc: 0.5924 - val_loss: 0.9876 - val_acc: 0.6

1562/1562 [==============================] - 163s - loss: 0.7434 - acc: 0.7506 - val_loss: 0.6338 - val_acc: 0.7888
Epoch 64/200
1562/1562 [==============================] - 116s - loss: 0.7431 - acc: 0.7514 - val_loss: 0.6231 - val_acc: 0.7926
Epoch 65/200
1562/1562 [==============================] - 142s - loss: 0.7414 - acc: 0.7520 - val_loss: 0.5990 - val_acc: 0.7949
Epoch 66/200
1562/1562 [==============================] - 147s - loss: 0.7420 - acc: 0.7508 - val_loss: 0.6202 - val_acc: 0.7906
Epoch 67/200
1562/1562 [==============================] - 139s - loss: 0.7379 - acc: 0.7519 - val_loss: 0.6581 - val_acc: 0.7852
Epoch 68/200
1562/1562 [==============================] - 130s - loss: 0.7397 - acc: 0.7534 - val_loss: 0.6729 - val_acc: 0.7748
Epoch 69/200
1562/1562 [==============================] - 119s - loss: 0.7419 - acc: 0.7516 - val_loss: 0.6187 - val_acc: 0.7915
Epoch 70/200
1562/1562 [==============================] - 128s - loss: 0.7411 - acc: 0.7518 - val_loss: 0.6259

1562/1562 [==============================] - 141s - loss: 0.8019 - acc: 0.7369 - val_loss: 0.7112 - val_acc: 0.7631
Epoch 127/200
1562/1562 [==============================] - 134s - loss: 0.8007 - acc: 0.7380 - val_loss: 0.6932 - val_acc: 0.7673
Epoch 128/200
1562/1562 [==============================] - 125s - loss: 0.8062 - acc: 0.7379 - val_loss: 0.7745 - val_acc: 0.7366
Epoch 129/200
1562/1562 [==============================] - 135s - loss: 0.8081 - acc: 0.7347 - val_loss: 0.8226 - val_acc: 0.7346
Epoch 130/200
1562/1562 [==============================] - 130s - loss: 0.8147 - acc: 0.7338 - val_loss: 0.7076 - val_acc: 0.7603
Epoch 131/200
1562/1562 [==============================] - 133s - loss: 0.8209 - acc: 0.7319 - val_loss: 0.6917 - val_acc: 0.7783
Epoch 132/200
1562/1562 [==============================] - 129s - loss: 0.8146 - acc: 0.7347 - val_loss: 0.6947 - val_acc: 0.7815
Epoch 133/200
1562/1562 [==============================] - 134s - loss: 0.8187 - acc: 0.7342 - val_loss:

1562/1562 [==============================] - 128s - loss: 1.1296 - acc: 0.6426 - val_loss: 1.1455 - val_acc: 0.6232
Epoch 189/200
1562/1562 [==============================] - 132s - loss: 1.1396 - acc: 0.6378 - val_loss: 1.1179 - val_acc: 0.6249
Epoch 190/200
1562/1562 [==============================] - 126s - loss: 1.1343 - acc: 0.6387 - val_loss: 1.0670 - val_acc: 0.6938
Epoch 191/200
1562/1562 [==============================] - 131s - loss: 1.1553 - acc: 0.6325 - val_loss: 1.0719 - val_acc: 0.6253
Epoch 192/200
1562/1562 [==============================] - 131s - loss: 1.1679 - acc: 0.6279 - val_loss: 1.1734 - val_acc: 0.5959
Epoch 193/200
1562/1562 [==============================] - 141s - loss: 1.1657 - acc: 0.6327 - val_loss: 1.1382 - val_acc: 0.6252
Epoch 194/200
1562/1562 [==============================] - 128s - loss: 1.1883 - acc: 0.6232 - val_loss: 1.1176 - val_acc: 0.6717
Epoch 195/200
1562/1562 [==============================] - 118s - loss: 1.1900 - acc: 0.6221 - val_loss: